# Makam pitch sequence classification with LSTM

### Library importing for file reading and preprocessing

### Preprocessing pitch files

Before proceeding, the pitch files on the CompMusic Dunya makam corpus need to be converted in the quantized pitch series encoding constructed as described in the pseudocode below.

In [1]:
import glob
import os
import numpy as np
from keras.preprocessing import sequence
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


### File reading, octave folded encoding

In [2]:
#quantized octave-folded pitch file directory
ofq_read_dir = "./octfold_qdata/" 

#Makam list for more efficient file searching during label retrieval
makams = ["Acemasiran", "Acemkurdi", "Bestenigar", "Beyati", "Hicaz", "Hicazkar", "Huseyni", "Huzzam", "Karcigar", "Kurdilihicazkar", "Mahur", "Muhayyer", "Neva", "Nihavent", "Rast", "Saba", "Segah", "Sultaniyegah", "Suzinak", "Ussak"]

all_qpitch = [] #array holding all quantized pitch series as strings
y = [] #holds makam labels
max_length = 0
for root, dirs, files in os.walk(ofq_read_dir):
    for name in files:
        if '.pitch' in name:
            #retrieve label from parent of original path
            for makam in makams:
                if (os.path.isfile("./otmm_makam_recognition_dataset/data/" + makam + "/" + name) == True):
                    y.append(makams.index(makam))
                    break
            
            with open(os.path.join(root, name)) as f:
                content = f.read()
                all_qpitch.append(content)
print("File number:", len(y))
print("Array length:", len(all_qpitch))

File number: 1000
Array length: 1000


### Preprocessing
Pading input sequences

In [4]:
#Tokenization
top_k = 636 #53*4*3
tokenizer = Tokenizer(num_words=top_k, split='\n')
tokenizer.fit_on_texts(all_qpitch)
seqs = tokenizer.texts_to_sequences(all_qpitch)
print(seqs)

In [7]:
#Padding
X = sequence.pad_sequences(seqs, padding='post')
print(X)
print(len(X))

[[15  5  2 ...  0  0  0]
 [11  7  2 ...  0  0  0]
 [47  4  1 ...  0  0  0]
 ...
 [12  8  1 ...  0  0  0]
 [14  7  1 ...  0  0  0]
 [33  4  1 ...  0  0  0]]
1000


### Library importing for deep learning

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

### Train - Test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Building the LSTM

In [10]:
#Word embedding
embedding_vector_len = 7 #max pitch value length: 4, comma, significance value of length 1, newline 

#Network topology
model = Sequential()

model.add(Embedding(top_k, embedding_vector_len, input_length=len(X[0])))
model.add(Dropout(0.2))
model.add(LSTM(20))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [11]:
#Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 213393, 7)         4452      
_________________________________________________________________
dropout_1 (Dropout)          (None, 213393, 7)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                2240      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 6,713
Trainable params: 6,713
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Fitting
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

Epoch 1/10


In [ ]:
#Model evaluation
scores = model.evaluate(X_test, y_test, verbose=1)
print(scores)